In [1]:
import numpy as np
import pandas as pd
import loadData
import monteCarloClass
import time

In [4]:
import os
import json
import glob
import pandas as pd
from pathlib import Path

def load_all_model_results():
    """Load all model fit results from the model_fits directory"""
    results = []
    
    # Find all JSON files in model_fits directory
    json_files = glob.glob("model_fits/**/*.json", recursive=True)

    
    
    for file_path in json_files:
        # if in filename "all_all", skip
        if "all" in os.path.basename(file_path):
            continue
        try:
            with open(file_path, 'r') as f:
                result = json.load(f)
            
            # Extract participant ID and model type from filename if not in data
            filename = os.path.basename(file_path)
            parts = filename.replace('.json', '').split('_')
            #print(parts)
            if 'participantID' not in result:
                result['participantID'] = parts[0]

            
            result['modelType'] = parts[1]+"_"+parts[3]
            
            if parts[2]!="LapseFree":
                #print("Lapse fixed model, skipping")
                continue
            

            results.append(result)
            
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    return results

# Load all results
all_results = load_all_model_results()
print(f"Loaded {len(all_results)} model fit results")

# Convert to DataFrame for easier analysis
df_results = pd.DataFrame(all_results)
print(f"Participants: {sorted(df_results['participantID'].unique())}")
print(f"Models: {sorted(df_results['modelType'].unique())}")


Loaded 132 model fit results
Participants: ['as', 'dt', 'hh', 'ip', 'ln1', 'ln2', 'mh', 'ml', 'mt', 'oy', 'qs', 'sx']
Models: ['fusionOnlyLogNorm_sharedPrior', 'fusionOnly_sharedPrior', 'gaussian_sharedPrior', 'logLinearMismatch_sharedPrior', 'lognorm_sharedPrior', 'probabilityMatchingLogNorm_sharedPrior', 'probabilityMatching_sharedPrior', 'selection_sharedPrior', 'switchingFree_sharedPrior', 'switchingWithConflict_sharedPrior', 'switching_sharedPrior']


In [3]:
df_results

,participantID,modelType,fittedParams,AIC,BIC,logLikelihood,n_conditions
0,sx,switchingFree_sharedPrior,"[0.0005876257007729965, 0.08356170619715197, 0...",2140.309309,2174.759511,-1062.154655,548
1,sx,lognorm_sharedPrior,"[0.0010000000743195236, 0.29345128405839205, 0...",2155.610053,2194.366530,-1068.805026,548
2,sx,logLinearMismatch_sharedPrior,"[0.000777923726726774, 0.293559182663938, 0.29...",2169.746267,2208.065275,-1075.873133,522
3,sx,fusionOnly_sharedPrior,"[0.022887993079794316, 0.153758934479332, 0.16...",2152.265325,2177.811330,-1070.132662,522
4,sx,fusionOnlyLogNorm_sharedPrior,"[0.001000086629595498, 0.30521380870952497, 0....",2149.645484,2175.483135,-1068.822742,548
...,...,...,...,...,...,...,...
139,ip,switching_sharedPrior,"[0.27314112638435994, 0.10224985842073724, 0.2...",2463.951618,2501.756200,-1222.975809,493
140,ip,gaussian_sharedPrior,"[0.23508676538004994, 0.09138144981005791, 0.3...",2448.722277,2486.526859,-1215.361138,493
141,ip,logLinearMismatch_sharedPrior,"[0.1716759013107659, 0.23309593249388888, 1.06...",2415.656094,2453.460677,-1198.828047,493
142,ip,fusionOnlyLogNorm_sharedPrior,"[0.13788956496247556, 0.23585898570927377, 1.1...",2404.078456,2429.281511,-1196.039228,493


In [32]:
participantIds=sorted(df_results['participantID'].unique())
modelTypes=sorted(df_results['modelType'].unique())
nParticipants=len(participantIds)
nModels=len(modelTypes)

import fitSaver
import os

def loadSimulatedData(mc_fitter, dataName,participantID, modelType=None):
    participantID = dataName.split(".csv")[0]
    #modelType = mc_fitter.modelName

    if mc_fitter.sharedLambda:
        modelType += "_LapseFix"
    else:
        modelType += "_LapseFree"

    if mc_fitter.freeP_c:
        modelType += "_contextualPrior"
    else:
        modelType += "_sharedPrior"

    filename = f"{participantID.split('_')[0]}_{modelType}_simulated.csv"
    filename = os.path.join("simulated_data",participantID.split('_')[0], filename)
    try:
        
        simulatedData= pd.read_csv(filename)
        print(f"Loaded saved simulated data from {filename}")
        return simulatedData
    except:
        print(f"No saved simulated data found at {filename}")
        fitSaver.saveSimulatedData(mc_fitter, mc_fitter.dataName)
        mc_fitter.simulatedData= pd.read_csv(filename)
        print(f"Simulated data saved to {filename}")
        return mc_fitter.simulatedData
#proceed to simulate and save to {filename}
# eg., mc_fitter.simulatedData=loadSimulatedData(mc_fitter, mc_fitter.dataName, participantID=dataName.split(".csv")[0], modelType=mc_fitter.modelName)

# for each participant load simulated data and fit to psychometric functions and save the fitted params in psychometric_fits_simulated/participantID/participantID_modelType_psychometricFits.json
# no need to plot for now, just save the fitted params
import monteCarloClass

for participantID in participantIds:
    for modelType in modelTypes:
        row = df_results[(df_results['participantID'] == participantID) & (df_results['modelType'] == modelType)]
        sharedPrior = 'sharedPrior' in modelType
        modelType=modelType.split("_")[0]
        print(f"Processing participant {participantID}, model {modelType}, sharedPrior: {sharedPrior}")
        if row.empty:
            print(f"No results found for participant {participantID} and model {modelType}, skipping.")
            continue
        result = row.iloc[0].to_dict()
        
        data,dataName=loadData.loadData(participantID+"_all.csv")
        # Initialize monteCarloClass with the model parameters
        tmp_mc_fitter = monteCarloClass.OmerMonteCarlo(data)
        tmp_mc_fitter.modelName = modelType
        tmp_mc_fitter.sharedLambda = 'LapseFix' in modelType
        tmp_mc_fitter.freeP_c = not sharedPrior
        tmp_mc_fitter.dataName = dataName

        
        simulatedData = loadSimulatedData(mc_fitter=tmp_mc_fitter, dataName=tmp_mc_fitter.dataName,participantID=participantID, modelType=tmp_mc_fitter.modelName)
        tmp_mc_fitter.simulatedData = simulatedData
        tmp_mc_fitter.simDataFit=tmp_mc_fitter.fitMultipleStartingPoints(tmp_mc_fitter.simulatedData,1)
        # save to psychometric_fits_simulated/participantID/participantID_modelType_psychometricFits.json without fitSaver function
        save_dir = os.path.join("psychometric_fits_simulated", participantID.split('_')[0])
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{participantID}_{modelType}_psychometricFits.json")
        print(f"Saving psychometric fits for simulated data to {save_path}")
        fit_data = {
            'participantID': participantID,
            'modelType': modelType,
            'fitParams': tmp_mc_fitter.simDataFit.x.tolist(),  # Convert ndarray to list
        }
        # serialize to json
        with open(save_path, 'w') as f:
            json.dump(fit_data, f)






        
        

        



   

Processing participant as, model fusionOnlyLogNorm, sharedPrior: True

 Total trials before cleaning
: 2156
uniqueSensory: [1.2 0.1] 
 uniqueStandard: [0.5] 
 uniqueConflict: [np.float64(-0.25), np.float64(-0.17), np.float64(-0.08), np.float64(0.0), np.float64(0.08), np.float64(0.17), np.float64(0.25)]
total trials after cleaning: 2155
Created logDurRatio variable: range [-2.997, 0.643]
  → This represents log(test/standard) for Weber's law compliance
✓ Configuration validated: gaussian, sharedLambda=False, freeP_c=False
Loaded saved simulated data from simulated_data/as/as_fusionOnlyLogNorm_LapseFree_sharedPrior_simulated.csv
Created logDurRatio variable: range [-3.157, 0.646]
  → This represents log(test/standard) for Weber's law compliance
Saving psychometric fits for simulated data to psychometric_fits_simulated/as/as_fusionOnlyLogNorm_psychometricFits.json
Processing participant as, model fusionOnly, sharedPrior: True

 Total trials before cleaning
: 2156
uniqueSensory: [1.2 0.1] 

KeyboardInterrupt: 